Pocetne funkcije

In [4]:
import numpy as np
import cv2
import time
from matplotlib import pyplot as plt
#import image_analysis


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
def image_bin(image_gs):
    ret,image_bin = cv2.threshold(image_gs, 100, 255, cv2.THRESH_BINARY)
    return image_bin
def image_bin_adaptive(image_gs):
    image_bin = cv2.adaptiveThreshold(image_gs, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 10)
    return image_bin
def dilate(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.dilate(image, kernel, iterations=1)
def erode(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.erode(image, kernel, iterations=1)
def remove_noise(binary_image):
    ret_val = erode(dilate(binary_image))
    ret_val = invert(ret_val)
    return ret_val
def invert(image):
    return 255-image

Funkcija za crtanje pravougaonika

In [5]:
def draw_hand_rect(self, frame):  
    rows,cols,_ = frame.shape

    self.hand_row_nw = np.array([6*rows/20,6*rows/20,6*rows/20,10*rows/20,10*rows/20,10*rows/20,14*rows/20,14*rows/20,14*rows/20])

    self.hand_col_nw = np.array([9*cols/20,10*cols/20,11*cols/20,9*cols/20,10*cols/20,11*cols/20,9*cols/20,10*cols/20,11*cols/20])

    self.hand_row_se = self.hand_row_nw + 10
    self.hand_col_se = self.hand_col_nw + 10

    size = self.hand_row_nw.size
    for i in xrange(size):
        cv2.rectangle(frame,(self.hand_col_nw[i],self.hand_row_nw[i]),(self.hand_col_se[i],self.hand_row_se[i]),(0,255,0),1)
#        black = np.zeros(frame.shape, dtype=frame.dtype)
#        frame_final = np.hstack([black, frame])
    return frame

Kreiranje HSV histograma


In [6]:
def set_hand_hist(self, frame):  
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    roi = np.zeros([90,10,3], dtype=hsv.dtype)

    size = self.hand_row_nw.size
    for i in xrange(size):
        roi[i*10:i*10+10,0:10] = hsv[self.hand_row_nw[i]:self.hand_row_nw[i]+10, self.hand_col_nw[i]:self.hand_col_nw[i]+10]
        #self.hand_hist = cv2.calcHist([roi],[0, 1], None, [180, 256], [0, 180, 0, 256])
        self.hand_hist = cv2.calcHist([roi],[0],None,[256],[0,256])
    cv2.normalize(self.hand_hist, self.hand_hist, 0, 255, cv2.NORM_MINMAX)



Detekcija koze

In [7]:
def apply_hist_mask(frame, hist):  
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0,1], hist, [0,180,0,256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 100, 255, 0)
    thresh = cv2.merge((thresh,thresh, thresh))

    cv2.GaussianBlur(dst, (3,3), 0, dst)

    res = cv2.bitwise_and(frame, thresh)
    return res

Iscrtavanje ruke

In [8]:
def draw_final(self, frame, hand_detection):  
    hand_masked = image_analysis.apply_hist_mask(frame, self.hand_hist)
    #hand_masked = cv2.bitwise_and(frame,frame,hand_detection)
    contours = None # image_analysis.contours(hand_masked)
    if contours is not None and len(contours) > 0:
        max_contour = image_analysis.max_contour(contours)
        hull = image_analysis.hull(max_contour)
        centroid = image_analysis.centroid(max_contour)
        defects = image_analysis.defects(max_contour)

        if centroid is not None and defects is not None and len(defects) > 0:   
            farthest_point = image_analysis.farthest_point(defects, max_contour, centroid)

            if farthest_point is not None:
                self.plot_farthest_point(frame, farthest_point)

Main

In [9]:
cap = cv2.VideoCapture(1)

class self(object):
    hand_row_nw = np.array((1,6))
    hand_col_nw = np.array((1,6))
    hand_row_se = np.array((1,6))
    hand_col_se = np.array((1,6))

    
while(1):
    ret, frame = cap.read()
    
    gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur= cv2.GaussianBlur(gray,(5,5),0)
    ret,thresh1 = cv2.threshold(blur,70,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#    cv2.imshow('sa',thresh1)
    frame=draw_hand_rect(self,frame)

   
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('b'):
        break
    
set_hand_hist(self,frame)

#print self.hand_row_nw
#plt.hist(self.hand_hist.ravel(),256,[0,256]); plt.show()
#color = ('b','g','r')
#for i,col in enumerate(color):
#    histr = cv2.calcHist([frame],[i],None,[256],[0,256])
#    plt.plot(self.hand_hist,color = col)
#    plt.xlim([0,256])
#plt.show()   
    
    

while(1):
    ret, frame = cap.read()

    res=apply_hist_mask(frame, self.hand_hist) 
    draw_final(self, frame,res) 
    cv2.imshow('as',res)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()

error: C:\builds\master_PackSlaveAddon-win64-vc12-static\opencv\modules\imgproc\src\color.cpp:7564: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor
